In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint
import json
from time import sleep
from datetime import date
import zipfile
import os
import seaborn as sns

# Incorporated citipy to determine city based on latitude and longitude
# pip install citipy
from citipy import citipy

# Output File location
output_folder = "output_data"

In [2]:
# Open the zip file:
working_dir = os.getcwd()
last_slash = working_dir.rfind('\\')
print(last_slash)
print(working_dir[last_slash+1:])

if working_dir[last_slash+1:].startswith("Walking-In-Paris"):
    csvpath = os.path.join("source_data", "detail_listings.zip")
else:
    csvpath = os.path.join("..", "source_data", "detail_listings.zip")
    
zf = zipfile.ZipFile(csvpath);

# zf = zipfile.ZipFile('../source_data/detail_listings.zip') 
df = pd.read_csv(zf.open('detail_listings.csv'), low_memory=False)

rows_in_df = len(df)
print(f'Data set has {rows_in_df} rows')

# Drop unneeded columns:
df = df.drop(columns = [
    "listing_url", 
    "thumbnail_url", 
    "picture_url", 
    "medium_url", 
    "xl_picture_url", 
    "host_url", 
    "host_thumbnail_url", 
    "host_picture_url", 
    "scrape_id",
    "host_listings_count", # There is a host_total_listings_count that is a dupe
    "host_acceptance_rate", # NaN
    "calendar_last_scraped",
    "bed_type",
    "last_scraped",
    "calendar_updated",
    "has_availability",
    "availability_30", 
    "availability_60", 
    "availability_90", 
    "availability_365", 
    "license"
])

df.head(1)

55
Walking-In-Paris (1)
Data set has 23745 rows


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5065,MAUKA BB,"Perfect for your vacation, Staycation or just ...",Mauka B & B is 400 square feet studio with a p...,"Perfect for your vacation, Staycation or just ...",none,Neighbors here are friendly but are not really...,Located near by is Kalopa State Park with 100 ...,There is a bus stop located in Honoka'a town ...,A BBQ grill is available,...,f,f,moderate,f,f,2,2,0,0,0.45


In [3]:
# Group the columns 
        
print('')
print('----------------------')
print('Property-specific columns:')
print('----------------------')
# These are wildcard search terms:
cols = ["experiences_offered", "street", "city", "state", "zipcode", "market", "country", "property"
    , "location", "accomodate", "room", "guest", "is_business_travel_ready", "instant_bookable", "cancel"]
for col in df.columns: 
    if col in cols:
        print(col) 
        
print('')
print('----------------------')
print('All columns:')
print('----------------------')
for col in df.columns: 
    print(col) 


----------------------
Property-specific columns:
----------------------
experiences_offered
street
city
state
zipcode
market
country
instant_bookable
is_business_travel_ready

----------------------
All columns:
----------------------
id
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
host_id
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_is_superhost
host_neighbourhood
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
m

In [4]:
# Create a dataframe w 1 row for each column in the source:
data_dict = pd.DataFrame(columns = [
    "ColumNumber"
    , "Column"
    , "Group"
    , "Subgroup"
    , "Notes"
])

for column in df:
    data_dict = data_dict.append({
            "Column": column
            , "Group": ""
            , "Subgroup": ""
            , "Notes": ""
        }
        , ignore_index=True
    )

# Make the index the column #
data_dict['ColumNumber'] = data_dict.index
        
# For each column, add context:
for index, row in data_dict.iterrows():
    val = str(row["Column"])
    
    if val == "id":
        data_dict.loc[index, "Notes"] = "Primary Key"
    
    if val.find("host") > -1: # val.startswith("host_"):
        data_dict.loc[index, "Group"] = "Host-related"
    
    if val.find("review") > -1:
        data_dict.loc[index, "Group"] = "Review-related"
        
    if (val.find("rule") > -1) | (val.find("require") > -1):
        data_dict.loc[index, "Group"] = "Requirements-related"
        
    if (val.find("neighbo") > -1):
        data_dict.loc[index, "Group"] = "Neighborhood-related"
        
    if (val.find("nights") > -1):
        data_dict.loc[index, "Group"] = "Booking-related"
        
    # Property specific:
    cols = [
        "property", "room", "guest", "is_business_travel_ready", "instant_bookable", "cancel"
        , "space", "name", "summary", "description", "notes", "transit", "access", "interaction"
        , "property_type", "room_type", "amenities"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        
    # Property - "Location" specific:
    cols = [
        "location", "street", "city", "state", "zipcode", "market", "country"
        , "latitude", "longitude", "is_location_exact"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Location"
        
    # Property - "Interior" specific:
    cols = [
        "bathrooms", "bedrooms", "beds", "square_feet"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Interior"
        
    # Property - "Pricing" specific:
    # Things about price, or that we might associate w price
    cols = [
        "price", "weekly_price", "monthly_price"        
        , "security_deposit", "cleaning_fee"
        , "accomodates", "guests_included", "extra_people"
        , "experiences_offered"
    ]
    if any(word in val for word in cols):
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Pricing"
        
    # Had to hard code this one for some reason:
    if val == "accommodates":
        data_dict.loc[index, "Group"] = "Property-related"
        data_dict.loc[index, "Subgroup"] = "Pricing"
        
print(data_dict.groupby(["Group", "Subgroup"])["Column"].count())
print('')
print('Columns without a group:')
data_dict[data_dict["Group"] == ""]

Group                 Subgroup
                                   1
Booking-related                    8
Host-related                      12
Neighborhood-related               5
Property-related                  20
                      Interior     4
                      Location    13
                      Pricing      9
Requirements-related               2
Review-related                    11
Name: Column, dtype: int64

Columns without a group:


,ColumNumber,Column,Group,Subgroup,Notes
0,0,id,,,Primary Key


In [5]:
# CLEANUP 

# Drop rows with number_of_reviews = 0
# Get names of indexes for which column number_of_reviews has value 0
# Drop by index:
indexNames = df[df['number_of_reviews'] == 0 ].index
df.drop(indexNames , inplace=True)

# Drop rows with number_of_reviews = NaN
df.dropna(axis=0, subset=('number_of_reviews', ))

# Drop rows with NaN review data
df.dropna(axis=0, subset=(
    'review_scores_rating', 
    'review_scores_accuracy', 
    'review_scores_cleanliness', 
    'review_scores_checkin', 
    'review_scores_communication', 
    'review_scores_location', 
    'review_scores_value'
))

len(df)

18291

In [6]:
# Strip the $ out of the pricing-based columns
data_dict[(data_dict["Group"] == "Property-related") & (data_dict["Subgroup"] == "Pricing")]

if df["price"].dtype != "float64":
    df["price"] = df["price"].str.replace("$", "")
    df["price"] = df["price"].str.replace(",", "")
    df["price"] = df["price"].astype("float64")

if df["weekly_price"].dtype != "float64":
    df["weekly_price"] = df["weekly_price"].str.replace("$", "")
    df["weekly_price"] = df["weekly_price"].str.replace(",", "")
    df["weekly_price"] = df["weekly_price"].astype("float64")

if df["monthly_price"].dtype != "float64":
    df["monthly_price"] = df["monthly_price"].str.replace("$", "")
    df["monthly_price"] = df["monthly_price"].str.replace(",", "")
    df["monthly_price"] = df["monthly_price"].astype("float64")
    
if df["security_deposit"].dtype != "float64":
    df["security_deposit"] = df["security_deposit"].str.replace("$", "")
    df["security_deposit"] = df["security_deposit"].str.replace(",", "")
    df["security_deposit"] = df["security_deposit"].astype("float64")
    
if df["cleaning_fee"].dtype != "float64":
    df["cleaning_fee"] = df["cleaning_fee"].str.replace("$", "")
    df["cleaning_fee"] = df["cleaning_fee"].str.replace(",", "")
    df["cleaning_fee"] = df["cleaning_fee"].astype("float64")   

if df["extra_people"].dtype != "float64":
    df["extra_people"] = df["extra_people"].str.replace("$", "")
    df["extra_people"] = df["extra_people"].str.replace(",", "")
    df["extra_people"] = df["extra_people"].astype("float64")

In [7]:
df.head()

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5065,MAUKA BB,"Perfect for your vacation, Staycation or just ...",Mauka B & B is 400 square feet studio with a p...,"Perfect for your vacation, Staycation or just ...",none,Neighbors here are friendly but are not really...,Located near by is Kalopa State Park with 100 ...,There is a bus stop located in Honoka'a town ...,A BBQ grill is available,...,f,f,moderate,f,f,2,2,0,0,0.45
1,5269,Upcountry Hospitality in the 'Auwai Suite,"The 'Auwai Suite is a lovely, self-contained a...","The ‘Auwai Suite is a lovely, self-contained a...","The 'Auwai Suite is a lovely, self-contained a...",none,"We are located on the ""sunny side"" of Waimea, ...",NaN,One parking spot is provided with your stay.,You'll have access to your private garden suit...,...,t,f,strict_14_with_grace_period,f,f,5,4,1,0,0.09
2,5387,Hale Koa Studio & 1 Bedroom Units!!,This Wonderful Spacious Studio apt/flat is in ...,"Hale Koa Duplex: This duplex has a wonderful, ...",This Wonderful Spacious Studio apt/flat is in ...,none,IN a Farm belt area with small commercial farm...,tropical setting with view and easy access to ...,"Close to Major highway (belt road) accessable,...","entire apartment, with full kitchen, baths an...",...,t,f,strict_14_with_grace_period,f,f,3,3,0,0,1.36
3,5389,Keauhou Villa,It is less than 10 minute walk to the Keauhou ...,"Location: Keauhou, Kona Coast, Big Island, Haw...",It is less than 10 minute walk to the Keauhou ...,none,It is less than 10 minute walk to the Keauhou ...,It is less than 10 minute walk to the Keauhou ...,It is less than 10 minute walk to the Keauhou ...,"Entire House with Garage, Screened Lanai, Fron...",...,f,f,flexible,f,f,3,3,0,0,0.23
4,5390,STAY AT PRINCE KUHIO!,NaN,"Prince Kuhio, Studio Unit. Sleeps 2 privately....","Prince Kuhio, Studio Unit. Sleeps 2 privately....",none,NaN,NaN,NaN,NaN,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,1.01
